In [61]:
import plotly.graph_objects as go
import numpy as np
import random
import copy
import time

In [62]:
min_range = -2
max_range = 2

## **Class implementation**

In [63]:
# min at (0, -1)
def goldstein_price(individual: list[float, float]) -> float:
	x, y = individual
	term1 = 1 + (x + y + 1) ** 2 * (19 - 14 * x + 3 * x ** 2 - 14 * y + 6 * x * y + 3 * y ** 2)
	term2 = 30 + (2 * x - 3 * y) ** 2 * (18 - 32 * x + 12 * x ** 2 + 48 * y - 36 * x * y + 27 * y ** 2)
	return term1 * term2

def fitness_goldstein_price(individual: list[float, float]):
	return 1/(1 + goldstein_price(individual) - 3.0)

In [64]:
class AG_Goldstein_Price():
    def __init__(self, population_size: int):
        self.population = [[random.uniform(min_range, max_range) for _ in range(2)] for _ in range(population_size)]
        self.fitnesses = []

    def sort_pop(self, fitness_function, reverse_sort: bool) -> tuple[list[list], list]:
        """Sort population by fitness function. Return tuple with population list and fitness list"""

        fitness_list = [fitness_function(ind) for ind in self.population]
        lista = sorted(zip(self.population, fitness_list), key=lambda x: x[1], reverse=reverse_sort)
        self.population = [x[0] for x in lista]
        self.fitnesses = [x[1] for x in lista]

    def select(self, T: int) -> list[float, float]:
        """Return a copy of an indivudual by tournament selection. Population already ordered by fitness"""

        choices=random.choices(copy.copy(self.population), k=T)
        indices=[self.population.index(c) for c in choices]
        return self.population[np.argmin(indices)]

    def crossover(self, parent1: list[float, float], parent2: list[float, float], pcross: float) -> tuple[list,list]: 
        if random.random()<pcross:
            child1 = [parent1[0], parent2[1]]
            child2 = [parent1[1], parent2[0]]
        else:
            child1, child2 = parent1[:], parent2[:]
        return child1, child2

    def mutate(self, individual: list[float, float], pmut: float) -> list[float, float]:
        if random.random()<pmut:
            individual = [random.uniform(min_range, max_range), random.uniform(min_range, max_range)]
        return individual

    def evolve(self, fitness_function, pmut=0.1, pcross=0.7, ngen=100, T=2, trace=50, reverse_sort=False, elitism=False) -> None:
        """Evolution procedure. Initial population already created"""

        for i in range(ngen):
            new_pop = []
            self.sort_pop(fitness_function, reverse_sort)
            if elitism:
                new_pop.append(self.population[0])
                new_pop.append(self.population[1])
            while len(new_pop) != 100:   
                individual1 = self.select(T)
                individual2 = self.select(T)
                child1, child2 = self.crossover(individual1, individual2, pcross)
                mutated1 = self.mutate(child1, pmut)
                mutated2 = self.mutate(child2, pmut)
                new_pop.append(mutated1)
                new_pop.append(mutated2)
                
            self.population = [*new_pop] # make a copy

            if i % trace == 0 or i == ngen-1: # en la última gen se ordena
                self.sort_pop(fitness_function, reverse_sort)
                print(f"Nº gen: {i}, Best ind: {self.population[0]}, Best fitness: {self.fitnesses[0]:.3f}")

## **Visual test**

In [65]:
def goldstein_price_test(x, y):
	term1 = (1 + (x + y + 1) ** 2 * (19 - 14 * x + 3 * x ** 2 - 14 * y + 6 * x * y + 3 * y ** 2))
	term2 = (30 + (2 * x - 3 * y) ** 2 * (18 - 32 * x + 12 * x ** 2 + 48 * y - 36 * x * y + 27 * y ** 2))
	return term1 * term2

def plot_goldstein_price(solution: list[float,float]):
    x = np.linspace(min_range, max_range, 100)
    y = np.linspace(min_range, max_range, 100)
    X, Y = np.meshgrid(x, y)
    Z = goldstein_price_test(X, Y)

    # surface plot
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Jet', showscale=False, opacity=0.8)])

    # global minimum
    global_min_x, global_min_y = 0, -1
    global_min_z = goldstein_price_test(global_min_x, global_min_y)
    fig.add_trace(go.Scatter3d(
        x=[global_min_x], y=[global_min_y], z=[global_min_z],
        mode='markers',
        marker=dict(size=6, color='purple', symbol='circle'),
        name=f'Global Minimum ({global_min_x:.3f},{global_min_y:.3f})'
    ))

    sol_x, sol_y = solution
    sol_z = goldstein_price_test(sol_x, sol_y) 
    fig.add_trace(go.Scatter3d(
        x=[sol_x], y=[sol_y], z=[sol_z],
        mode='markers',
        marker=dict(size=6, color='black', symbol='circle'),
        name=f'Solution ({sol_x:.3f}, {sol_y:.3f})'
    ))

    # Update layout
    fig.update_layout(
        title='Goldstein-Price Function',
        scene=dict(
            xaxis_title='X-axis',
            yaxis_title='Y-axis',
            zaxis_title='Z-axis'
        ),
        margin=dict(l=0, r=0, b=0, t=40),  # Adjust margins for better fit
        width=700,
        height=550,
        showlegend=True,
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        )
    )

    fig.show()
# ---------------------------------------
plot_goldstein_price([1.0,1.0])

## **---------------------------Tests---------------------------**

In [71]:
start = time.time()
genetic_algorithm = AG_Goldstein_Price(population_size=100) 
genetic_algorithm.evolve(fitness_function=fitness_goldstein_price, pmut=0.1, ngen=2000, T=6, trace=200, reverse_sort=True)
minutos, segundos = divmod(time.time()-start, 60)
print(f"*******Tiempo transcurrido: {int(minutos)} minutos y {segundos:.2f} segundos*******")

plot_goldstein_price(genetic_algorithm.population[0])

Nº gen: 0, Best ind: [0.1480775893085542, -0.9365970619176598], Best fitness: 0.158
Nº gen: 200, Best ind: [-0.003056243818468918, -1.033396673339324], Best fitness: 0.667
Nº gen: 400, Best ind: [-0.003056243818468918, -1.033396673339324], Best fitness: 0.667
Nº gen: 600, Best ind: [-0.003056243818468918, -1.033396673339324], Best fitness: 0.667
Nº gen: 800, Best ind: [-0.003056243818468918, -1.033396673339324], Best fitness: 0.667
Nº gen: 1000, Best ind: [-0.003056243818468918, -1.0018341236226345], Best fitness: 0.997
Nº gen: 1200, Best ind: [-0.003056243818468918, -1.0018341236226345], Best fitness: 0.997
Nº gen: 1400, Best ind: [-0.003056243818468918, -1.0018341236226345], Best fitness: 0.997
Nº gen: 1600, Best ind: [-0.003056243818468918, -1.0018341236226345], Best fitness: 0.997
Nº gen: 1800, Best ind: [-0.003056243818468918, -1.0018341236226345], Best fitness: 0.997
Nº gen: 1999, Best ind: [-0.003056243818468918, -1.0018341236226345], Best fitness: 0.997
*******Tiempo transcurri